In [ ]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
from environs import Env

from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import make_audience_conf, dict_from_nested_type
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user
from typing import List
import json

env = Env()
env.read_env('.env-vlab-vlab', override=True)
db_conf = get_db_conf(env)

In [24]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [ ]:
template_state = CampaignState(userinfo.token, get_api(env, userinfo.token), '282370023153169', 'template-ad-sets')

template_state.campaign_state

In [26]:
geo_shares = {'North Governorate': .14,
              'Beirut Governorate': .08,
              'South Governorate': .11,
              'Nabatieh Governorate': .15,
              'Mount Lebanon Governorate': .34 ,
              'Beqaa Governorate': .18}

age_shares = { 18: .25, 
               25: .25, 
               35: .25, 
               50: .25}

In [27]:
def hyphen_case(s):
    return s.replace(' ', '-').lower().strip()

def get_geo_name(g):
    return g['targeting']['geo_locations']['regions'][0]['name']

In [28]:
age_sets = [a for a in template_state.adsets if 'Age' in a['name']]
geo_sets = [a for a in template_state.adsets if 'Lebanon - GEO' in a['name']]
gender_sets = [a for a in template_state.adsets if 'Lebanon - Gender' in a['name']]

# weird hack to deal with messed up template that I can't change
# due to disabled ad account
gender_sets[0]['targeting']['genders'] = [2]

geo_sets = [(hyphen_case(get_geo_name(g)), geo_shares[get_geo_name(g)], g)
            for g in geo_sets]

age_sets = [(str(g['targeting']['age_min']), age_shares[g['targeting']['age_min']], g) 
              for g in age_sets]

gender_sets = [(str(g['targeting']['genders'][0]), 0.5, g) for g in gender_sets]


In [37]:
from itertools import product
from functools import reduce

from math import ceil

groups = [
    (['age_max', 'age_min'],
     age_sets),
    (['genders'], gender_sets),
    (['geo_locations'],
     geo_sets),
]

def format_group_product(group):
    conf = {k:v for keys, (name, share, conf) in group
            for k, v in conf['targeting'].items() 
            if k in keys}

    name = '-'.join([n for _, (n, _, _) in group])

    share = reduce(lambda a,b: a*b, [s for _, (_, s, _) in group])
    share = ceil(share * 5000)

    return (name, share, conf)

groups = [[(keys, s) for s in sets] for keys, sets in groups]
groups = list(product(*groups))
groups = [format_group_product(g) for g in groups]

In [38]:
def conf_for_export(conf):
    conf['geo_locations'] = conf['geo_locations'].export_all_data()
    return conf

groups = [(n,s,conf_for_export(c)) for n, s, c in groups]

In [40]:
images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}

# CREATING CONFS - MNM

In [41]:
# make creatives conf
from adopt.marketing import CreativeConf


In [42]:
USER = "mchatila@worldbank.org"
CAMPAIGN = "mena-vaccination"
CAMPAIGNID = get_campaigns_for_user(USER, db_conf)[0]["id"]

# create_campaign_for_user(USER, CAMPAIGN, db_conf)

In [ ]:
c = {'optimization_goal': 'LINK_CLICKS',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 400000.0,
     'min_budget': 100.0,
     'opt_window': 7*24,
     'end_date': '2021-02-03',
     'proportional': False,
     'page_id': '106141768050427',
     'instagram_id': None,
     'ad_account': '282370023153169',
     'ad_campaign': 'embed-mena-lebanon-1'}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [config._asdict()], db_conf)

In [ ]:
from adopt.marketing import make_audience_conf, dict_from_nested_type
import json

audiences = [
    {
        "name": "vlab-embed-mena-vaccination-respondents",
        "shortcodes": ["vaccwelcome", "vaccinationar", "vaccinationeng", "vaccinationfr"],
        "subtype": "CUSTOM"
    },
]


audience_confs = [make_audience_conf(c) for c in audiences]
confs = [dict_from_nested_type(a) for a in audience_confs]

create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

In [ ]:
from adopt.marketing import CreativeConf


def _creative_conf(name, image, body, headline, welcome_message, button_text, form):
    return {
        "name": name,
        "image": image['name'],
        "image_hash": image['hash'],
        "body": body,
        "link_text": headline,
        "welcome_message": welcome_message,
        "button_text": button_text,
        "form": form,
    }

image_confs = [
    ('vlab-embed-mena-e-gift-card-fr', 
     images['e-gift-card_5_square.jpg'], 
     'Partagez votre opinion et courez la chance de gagner une carte cadeau de 150 000 LBP de HiCart!', 
     'Partagez votre opinion',
     'Partagez votre opinion et courez la chance de gagner une carte cadeau de 150 000 LBP de HiCart cliquez sur commencer pour commencer!',
     'Commencer'),
    ('vlab-embed-mena-e-gift-card-en', 
     images['e-gift-card_5_square.jpg'], 
     'Share your opinion and get the chance to win a 150,000 LBP gift card from HiCart!', 
     'Share your opinion', 
     'Share your opinion and get the chance to win a 150,000 LBP gift card from HiCart click begin to start!', 
     'Begin'),
    ('vlab-embed-mena-e-gift-card-ar', 
     images['e-gift-card_5_square.jpg'], 
     'شارك رأيك واحصل على فرصة للفوز بقسيمة شرائية بقيمة 150000 ليرة من HiCart!', 
     'شارك رأيك',
     'شارك رأيك واحصل على فرصة للفوز بقسيمة شرائية بقيمة 150000 ليرة من HiCart انقر ابدأ لتبدء',
     'ابدأ'),
]

creatives = [CreativeConf(**_creative_conf(*c, 'vaccwelcome')) for c in image_confs]
confs = [c._asdict() for c in creatives]

create_campaign_confs(CAMPAIGNID, "creative", confs, db_conf)

In [43]:
def make_stratum(id_, quota, targeting):
    return { 'id': id_,
             'metadata': {'stratumid': id_, 'country': 'lebanon'},
             'facebook_targeting': targeting, 
             'creatives': ['vlab-embed-mena-e-gift-card-ar', 'vlab-embed-mena-e-gift-card-fr', 'vlab-embed-mena-e-gift-card-en'],
             'audiences': [],
             'excluded_audiences': ["vlab-embed-mena-vaccination-respondents"],
             'quota': quota,
             'shortcodes': ['vaccinationar', 'vaccinationfr', 'vaccinationeng'],
             'question_targeting': {
                 'op': 'and',
                 'vars': [
                     { 'op': 'equal',
                       'vars': [{'type': 'response', 'value': 'md:stratumid'},
                                {'type': 'constant', 'value': id_}]},
                     { 'op': 'answered',
                       'vars': [{'type': 'response', 'value': 'refugee'}]}
                 ]
             }}

In [ ]:
from adopt.marketing import make_stratum_conf

strata = [make_stratum(*g) for g in groups]
strata_data = [dict_from_nested_type(make_stratum_conf(c)) for c in strata]
create_campaign_confs(CAMPAIGNID, "stratum", strata_data, db_conf)

# TESTING

In [47]:
mal = load_basics(CAMPAIGNID, env)

In [48]:
%%time

from adopt.malaria import update_ads_for_campaign


instructions, report = update_ads_for_campaign(mal)

INFO:root:Loaded 48 adsets from campaign embed-mena-lebanon-1


INFO:root:Loaded 144 ads from campaign embed-mena-lebanon-1


CPU times: user 2min 19s, sys: 53 ms, total: 2min 19s
Wall time: 2min 56s


In [ ]:
mal.config

In [50]:
from adopt.malaria import days_left

days_left(mal.config)

1

In [60]:
import pandas as pd

pd.DataFrame([{'stratum': s, **v} for s, v in report.items()]).to_csv('lebanon-report.csv', index=False)

In [73]:
df = pd.DataFrame([{'stratum': s, **v} for s, v in report.items()])

df[df.remaining < 0].apply(lambda r: r.remaining * r.price, 1).sum()

-9199.797018536681

In [19]:
sum([r['goal'] for r in report.values()])

100000

In [52]:
sum([r['budget'] for r in report.values()])

272154

In [ ]:
from adopt.campaign_queries import create_adopt_report

db_conf['user'] = 'adopt'
create_adopt_report(CAMPAIGNID, 'FACEBOOK_ADOPT', report, db_conf)

In [ ]:
import pandas as pd
pd.DataFrame(report).T

In [ ]:
from adopt.malaria import run_instructions

run_instructions(instructions, mal.state)

In [21]:
from adopt.malaria import hydrate_strata, hydrate_audiences

In [ ]:
audience_confs = confs["audience"]

df = get_df(db_conf, userinfo.survey_user, audience_confs)

audiences = hydrate_audiences(df, m, audience_confs)

In [ ]:
strata = hydrate_strata(state, confs["stratum"], confs["creative"])
df = get_df(db_conf, userinfo.survey_user, strata)